# Model Project - OLG

In [2]:
import numpy as np
from scipy import optimize
import sympy as sm

import matplotlib.pyplot as plt
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2


# Model description



In this project we will first consider a an OLG model with a pay as you go system which we will later compare to an OLG model with a fully funded system.

Individuals live for two periods and we assume constant population:

$L_t=L_{t+1}$

The lifetime utility of a household is defined as:

$U_t=ln(c_{1t})+\frac{1}{(1+\rho)}ln(c_{2t+1}), \ \ \rho>-1$

This is subject to the following constraints:

$c_{1t}=w_t(1-\tau)-s_t$

$c_{2t+1}=s_t(1+r_{t+1})+\tau w_{t+1}$

Thus the model has the following parameters:

$L_t$: The population at time $t$

$U_t$: Lifetime household utility

$\rho$: Consumption discount rate

$c_{1t}$: Consumption of the young

$c_{2t+1}$: Consumption of the old

$w_t$: Wage

$r_t$: Return on capital

$s_t$: Savings

In [3]:
# Define symbols for sympy
K_t = sm.symbols('K_t')
L_t = sm.symbols('L_t')
A = sm.symbols('A')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
rho = sm.symbols('rho')
w_t = sm.symbols('w_t')
r_t = sm.symbols('r_t')
k_t = sm.symbols('k_t')
k_plus = sm.symbols('k_t+1')
Pi = sm.symbols('Pi_t')

s_t = sm.symbols('s_t')
U_t = sm.symbols('U_t')
U = sm.symbols('U_t')
r_plus = sm.symbols('r_t+1')
k = sm.symbols('k^*')

d_t = sm.symbols('d_t')
d_plus = sm.symbols('d_t+1')
tau = sm.symbols('tau')
s_payg = sm.symbols('s_payg')
w_plus = sm.symbols('w_t+1')
k_payg = sm.symbols('k_payg')
U_payg = sm.symbols('U_paygt')
kpaygss = sm.symbols('k_payg^*')

In [4]:
#In order to derive the wage (w) and return on capital (r) we need to solve the firms profit maximization problem.

#Define the profit function
profit = A*(K_t**alpha)*(L_t**(1-alpha))-w_t*L_t-(r_t+delta)*K_t

#Setup the FOC's
foc1=sm.diff(profit,K_t)
foc2=sm.diff(profit,L_t)

#Solve FOC's
solve_foc1=sm.solve(foc1,r_t)[0]
solve_foc2=sm.solve(foc2,w_t)[0]

#Print eqations
print('The profit of a given firm is defined as:')
display(sm.Eq(Pi,profit))
print('By taking the first order conditions to this, we get the expressions for w an r:')
display(sm.Eq(r_t,solve_foc1))
display(sm.Eq(w_t,solve_foc2))

The profit of a given firm is defined as:


Eq(Pi_t, A*K_t**alpha*L_t**(1 - alpha) - K_t*(delta + r_t) - L_t*w_t)

By taking the first order conditions to this, we get the expressions for w an r:


Eq(r_t, A*K_t**alpha*L_t*L_t**(-alpha)*alpha/K_t - delta)

Eq(w_t, -A*K_t**alpha*L_t**(-alpha)*(alpha - 1))

We now assume full capital depreciation $\delta=1$

In [5]:
#Setup equations
sample_r=alpha*A*k_t**(alpha-1)-1
sample_w=(1-alpha)*A*k_t**alpha

#Display them
print('By using the marginal products of capital and labour we can simplify these expressions to:')
display(sm.Eq(r_t,sample_r))
display(sm.Eq(w_t,sample_w))

By using the marginal products of capital and labour we can simplify these expressions to:


Eq(r_t, A*alpha*k_t**(alpha - 1) - 1)

Eq(w_t, A*k_t**alpha*(1 - alpha))

We now move unto the household optimization problem and determine the savings rate.

In [6]:
#We solve the household optimization problem and determine savings (s)

#Substitute constraints into utility function
U_t = sm.log(w_t*(1-tau)-s_t)+(1/(1+rho))*sm.log(s_t*(1+r_plus)+tau*w_plus)

#Take the FOC in regards to savings
FOC_U = sm.diff(U_t,s_t)

#Isolate savings
isolate_s = sm.solve(FOC_U,s_t)[0]

#Print results
print('Substitute the constraints into the utility function')
display(sm.Eq(U,U_t))
print('Take the FOC in regards to savings:')
display(sm.Eq(0,FOC_U))
print('Isolate savings:')
display(sm.Eq(s_t,isolate_s))
print('we can simplify this to:')
display(sm.Eq(s_t,-tau*w_plus*(rho+1)/((r_plus+1)*(rho+2))+w_t*(1-tau)/(rho+2)))


Substitute the constraints into the utility function


Eq(U_t, log(-s_t + w_t*(1 - tau)) + log(s_t*(r_t+1 + 1) + tau*w_t+1)/(rho + 1))

Take the FOC in regards to savings:


Eq(0, (r_t+1 + 1)/((rho + 1)*(s_t*(r_t+1 + 1) + tau*w_t+1)) - 1/(-s_t + w_t*(1 - tau)))

Isolate savings:


Eq(s_t, (-r_t+1*tau*w_t + r_t+1*w_t - rho*tau*w_t+1 - tau*w_t - tau*w_t+1 + w_t)/(r_t+1*rho + 2*r_t+1 + rho + 2))

we can simplify this to:


Eq(s_t, -tau*w_t+1*(rho + 1)/((r_t+1 + 1)*(rho + 2)) + w_t*(1 - tau)/(rho + 2))

# Steady state

We now wish to determine the steady state level of capital $k^*$

In [7]:
# We define capital accumulation using the previuos expressions and solve for the steady state level of capital.

#capital accumulation 
print('capital accumulation is given by')
display(sm.Eq(k_plus,-tau*w_plus*(rho+1)/((r_plus+1)*(rho+2))+w_t*(1-tau)/(rho+2)))
print('We insert the expressions for w and r and get')
display(sm.Eq(k_plus,(A*k_t**(alpha)*(1-alpha)*(1-tau))/(rho+2)-(k_plus**(1-alpha)*tau*w_plus*(rho+1))/(A*alpha*(rho+2))))

capital accumulation is given by


Eq(k_t+1, -tau*w_t+1*(rho + 1)/((r_t+1 + 1)*(rho + 2)) + w_t*(1 - tau)/(rho + 2))

We insert the expressions for w and r and get


Eq(k_t+1, A*k_t**alpha*(1 - alpha)*(1 - tau)/(rho + 2) - k_t+1**(1 - alpha)*tau*w_t+1*(rho + 1)/(A*alpha*(rho + 2)))

In the steady state it is true that 

$k_{t+1}=k_t=k^*$

We can therefore rewrite the above to:

In [8]:
display(sm.Eq(k,(1/(1+(1+rho)/(2+rho)*(1-alpha)/(alpha)*tau)*((1-alpha)*(1-tau))/(2+rho)*A)**(1/(1-alpha))))

Eq(k^*, (A*(1 - alpha)*(1 - tau)/((1 + tau*(1 - alpha)*(rho + 1)/(alpha*(rho + 2)))*(rho + 2)))**(1/(1 - alpha)))

In [9]:
#Define the steady state capital
ssk=sm.Eq(k_t,(1/(1+(1+rho)/(2+rho)*(1-alpha)/(alpha)*tau)*((1-alpha)*(1-tau))/(2+rho)*A)**-(1/(alpha-1)))
ssk_solve = sm.solve(ssk,k_t)

## Analytical solution

In [12]:
#Create a function to determine capital level in steady state given a set of parameters
ss_func = sm.lambdify((alpha,tau,rho,A),ssk_solve)




SyntaxError: unmatched ')' (<ipython-input-12-7c92fdc9a7da>, line 5)

In [18]:
def ss_alt(alpha, tau, rho, A):
    return((((A*(1-alpha)*(1-tau)))/(1+(tau*(1-alpha)*(rho+1))/(alpha*(rho+2))*(rho+2)))**(1/(1-alpha)))
print(f'The steady state in the pay as you go model is:')
ss_alt(1/4, 0.2, 0.05, 1)


The steady state in the pay as you go model is:


0.2638061168681807

In [474]:
# Assign arbitrary values to the parameters
value_alpha = 1/4
value_tau = 0.8
value_rho = 0.05
value_A = 1

#Run the function with the values
res=ss_func(value_alpha,value_tau,value_rho,value_A)

#Display results
display(sm.Eq(k,res))

SympifyError: SympifyError: [0.010509129538683937]

We caanot get this to work the way we want to using simpy, so we will have to do it manually.